<a href="https://colab.research.google.com/github/NickKornienko/Language-Identification-model/blob/main/LanguageUnderstanding_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_pickle('/content/drive/MyDrive/all_data.pkl')
df.head()

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,label
0,-368.843903,77.270210,-28.758459,28.591148,-9.293530,-6.967359,2.937174,0.106718,-2.853848,-5.559022,...,-5.811731,-2.380488,-1.242046,-3.116185,0.163225,-0.594025,0.902249,2.282130,2.089625,english
1,-292.091400,107.465469,-44.139664,44.164894,-17.887180,-12.471088,-3.590851,-7.044466,-7.811891,-6.014475,...,-5.738669,-5.255381,-1.789235,-2.641787,0.179766,2.768640,4.179280,6.194175,6.816234,english
2,-318.740234,95.526634,-39.401192,27.715158,-22.705025,-6.303998,-8.845734,-5.550812,-3.317510,-11.346023,...,-2.474380,-3.832747,-0.532228,0.773945,-2.555231,1.881754,1.472016,3.019983,5.518497,english
3,-315.532990,92.288322,-40.423939,34.508945,-18.966898,-4.724030,-8.200343,-17.788260,-7.930902,-5.146729,...,-3.247579,-5.742671,-3.982553,-1.573804,-2.270712,2.832158,2.902952,4.061225,5.034604,english
4,-301.566803,83.467590,-43.062504,54.350731,-23.995485,0.010661,-7.562351,-6.149078,-7.499183,-4.696762,...,-3.790015,-5.441263,-4.541253,-1.572427,-4.257127,0.672188,1.667025,4.608165,6.336764,english


In [ ]:
#let us
df['label'].value_counts()

french     75022
english    44035
spanish    36655
Name: label, dtype: int64

In [ ]:
df['label'].shape

(155712,)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Dropout, LSTM
from tensorflow.keras.utils import to_categorical

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

In [ ]:

# One-hot encode labels if they are categorical
# Create a label (category) encoder object
le = LabelEncoder()
df['label'] = le.fit_transform(df['label'])# Separate features and labels
X = df.drop('label', axis=1).values  # Features
y = df['label'].values  # Labels


num_classes = len(np.unique(y))
# One-hot encode labels
y = to_categorical(y, num_classes=num_classes)



# Reshape X to be 3D [samples, timesteps, features] for RNN
X = X.reshape((X.shape[0], 1, X.shape[1]))
print(X.shape)
print(y.shape)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(155712, 1, 40)
(155712, 3)
(124569, 1, 40)
(31143, 1, 40)
(124569, 3)
(31143, 3)


In [ ]:
X.shape

(155712, 1, 40)

In [ ]:
# Define RNN model
model = Sequential()
model.add(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(32))
model.add(Dense(3, activation='softmax'))  # Output layer

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/10
3893/3893 [==============================] - 36s 7ms/step - loss: 0.5904 - accuracy: 0.7617 - val_loss: 0.4646 - val_accuracy: 0.8173
Epoch 2/10
3893/3893 [==============================] - 29s 8ms/step - loss: 0.5047 - accuracy: 0.8021 - val_loss: 0.4260 - val_accuracy: 0.8360
Epoch 3/10
3893/3893 [==============================] - 27s 7ms/step - loss: 0.4831 - accuracy: 0.8105 - val_loss: 0.4147 - val_accuracy: 0.8418
Epoch 4/10
3893/3893 [==============================] - 26s 7ms/step - loss: 0.4695 - accuracy: 0.8163 - val_loss: 0.4073 - val_accuracy: 0.8444
Epoch 5/10
3893/3893 [==============================] - 27s 7ms/step - loss: 0.4603 - accuracy: 0.8212 - val_loss: 0.3893 - val_accuracy: 0.8526
Epoch 6/10
3893/3893 [==============================] - 28s 7ms/step - loss: 0.4507 - accuracy: 0.8248 - val_loss: 0.3830 - val_accuracy: 0.8553
Epoch 7/10
3893/3893 [==============================] - 28s 7ms/step - loss: 0.4454 - accuracy: 0.8279 - val_loss: 0.3705 - val_ac

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)

974/974 [==============================] - 4s 4ms/step - loss: 0.3602 - accuracy: 0.8643
Test Accuracy: 0.8643354773521423


In [ ]:
path = '/content/drive/MyDrive/255model1.h5'
model.save(path)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Define RNN model
model = Sequential()
model.add(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(32,return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(16))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))  # Output layer

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=16, validation_data=(X_test, y_test))


Epoch 1/10
7786/7786 [==============================] - 82s 9ms/step - loss: 0.6614 - accuracy: 0.7338 - val_loss: 0.5056 - val_accuracy: 0.8033
Epoch 2/10
7786/7786 [==============================] - 77s 10ms/step - loss: 0.5796 - accuracy: 0.7730 - val_loss: 0.4715 - val_accuracy: 0.8183
Epoch 3/10
7786/7786 [==============================] - 76s 10ms/step - loss: 0.5527 - accuracy: 0.7872 - val_loss: 0.4595 - val_accuracy: 0.8274
Epoch 4/10
7786/7786 [==============================] - 77s 10ms/step - loss: 0.5397 - accuracy: 0.7913 - val_loss: 0.4392 - val_accuracy: 0.8325
Epoch 5/10
7786/7786 [==============================] - 76s 10ms/step - loss: 0.5269 - accuracy: 0.7971 - val_loss: 0.4255 - val_accuracy: 0.8374
Epoch 6/10
7786/7786 [==============================] - 72s 9ms/step - loss: 0.5219 - accuracy: 0.7977 - val_loss: 0.4185 - val_accuracy: 0.8452
Epoch 7/10
7786/7786 [==============================] - 73s 9ms/step - loss: 0.5145 - accuracy: 0.8041 - val_loss: 0.4188 - va

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)

974/974 [==============================] - 5s 5ms/step - loss: 0.4036 - accuracy: 0.8505
Test Accuracy: 0.8504639863967896


In [ ]:
path = '/content/drive/MyDrive/255model_3layerLSTM.h5'
model.save(path)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Trying CNN

In [ ]:

#df['label'] = le.fit_transform(df['label'])# Separate features and labels
# Assuming 'df' is your DataFrame and 'label' is the column with labels
X = df.drop('label', axis=1).values
y = df['label'].values

# Reshape X to be [samples, time steps, features]
X_reshaped = X.reshape(X.shape[0], X.shape[1], 1)

# One-hot encode y if necessary
y_encoded = to_categorical(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y_encoded, test_size=0.2, random_state=42)


In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation, Conv1D, MaxPooling1D

In [ ]:
num_mfcc = 40  # Number of MFCC features
num_classes = 3  # Number of classes in your classification task

model = Sequential()
# Layer 1
model.add(Conv1D(32, 3, input_shape=(num_mfcc, 1)))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))

# Layer 2
model.add(Conv1D(64, 3))
model.add(Activation('relu'))

# Layer 3
model.add(Conv1D(128, 3))
model.add(Activation('relu'))

# Layer 4
model.add(Conv1D(256, 3))
model.add(Activation('relu'))

# Layer 5
model.add(Conv1D(512, 3))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))

# Flattening and Dense Layers
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))  # Output layer

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/10
3893/3893 [==============================] - 35s 7ms/step - loss: 0.3813 - accuracy: 0.8545 - val_loss: 0.2792 - val_accuracy: 0.8986
Epoch 2/10
3893/3893 [==============================] - 27s 7ms/step - loss: 0.2566 - accuracy: 0.9058 - val_loss: 0.2294 - val_accuracy: 0.9176
Epoch 3/10
3893/3893 [==============================] - 33s 8ms/step - loss: 0.2106 - accuracy: 0.9242 - val_loss: 0.2210 - val_accuracy: 0.9196
Epoch 4/10
3893/3893 [==============================] - 26s 7ms/step - loss: 0.1790 - accuracy: 0.9360 - val_loss: 0.2003 - val_accuracy: 0.9298
Epoch 5/10
3893/3893 [==============================] - 28s 7ms/step - loss: 0.1582 - accuracy: 0.9436 - val_loss: 0.1808 - val_accuracy: 0.9376
Epoch 6/10
3893/3893 [==============================] - 29s 7ms/step - loss: 0.1406 - accuracy: 0.9491 - val_loss: 0.1867 - val_accuracy: 0.9371
Epoch 7/10
3893/3893 [==============================] - 26s 7ms/step - loss: 0.1316 - accuracy: 0.9528 - val_loss: 0.1875 - val_ac

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)

974/974 [==============================] - 6s 6ms/step - loss: 0.2014 - accuracy: 0.9383
Test Accuracy: 0.938284695148468


In [ ]:
path = '/content/drive/MyDrive/255mode_5layerCNN.h5'
model.save(path)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# Make predictions
predictions = model.predict(X_test)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(y_test, axis=1)

# Generate classification report
report = classification_report(true_classes, predicted_classes)
print("Classification Report:\n", report)

# Generate confusion matrix
conf_matrix = confusion_matrix(true_classes, predicted_classes)
print("Confusion Matrix:\n", conf_matrix)


974/974 [==============================] - 5s 5ms/step
Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.92      0.91      8699
           1       0.94      0.95      0.95     15092
           2       0.97      0.93      0.95      7352

    accuracy                           0.94     31143
   macro avg       0.94      0.93      0.94     31143
weighted avg       0.94      0.94      0.94     31143

Confusion Matrix:
 [[ 8020   595    84]
 [  610 14367   115]
 [  263   255  6834]]
